# Импорт

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from LookGenerator.datasets.classes import PersonSegmentationDataset
from LookGenerator.networks.segmentation import UNet, train_unet

# Загрузка данных

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [3]:
batch_size=1

In [4]:
train_dataset = PersonSegmentationDataset("C:\\Users\\asus\\Desktop\\datasets\\VITONHD\\train", transform=transform, densepose=True, parse_agnostic=False, parse=False)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(len(train_dataset))

11647


In [5]:
val_dataset = PersonSegmentationDataset("C:\\Users\\asus\\Desktop\\datasets\\VITONHD\\test", transform=transform, densepose=True, parse_agnostic=False, parse=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
len(val_dataloader)

2032

In [13]:
data = None
targets = None
for X, y in train_dataloader:
    print(X.shape)
    print(y.shape)
    data = X
    targets = y
    break

torch.Size([1, 3, 1024, 768])
torch.Size([1, 3, 1024, 768])


# Обучение модели

In [6]:
model = UNet()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
device = 'gpu' if torch.cuda.is_available() else 'cpu'

In [ ]:
output = model(data)
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(output, targets)
print(loss.item())

In [7]:
train_history, val_history = train_unet(
    model,
    train_dataloader,
    val_dataloader,
    optimizer,
    device=device,
    epoch_num=5
)

RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 4